In [ ]:
import pandas_gbq

In [ ]:
df = pandas_gbq.read_gbq("SELECT * FROM `nais-analyse-prod-2dcc.deploys.deploys_prod_aggregated`", project_id='nais-analyse-prod-2dcc')

In [ ]:
#Filter out canaries and auto-deployed system apps
df = df[df['application'] != 'nais-deploy-canary']
df = df[df['application'] != 'testapp-storage']
df = df[df['application'] != 'kafkarator-canary']

In [ ]:
df['dato'] = df['deployed_timestamp'].dt.date
df['ukenr'] = df['deployed_timestamp'].dt.isocalendar().week.astype('str')
df['ukenr'] = df['ukenr'].apply( lambda x: x.zfill(2))
df['uke'] = df['deployed_timestamp'].dt.isocalendar().year.astype('str') + '-' + df['ukenr']
df['måned'] = df['deployed_timestamp'].dt.month
df['måned'] = df['måned'].astype('str').apply(lambda x: x.zfill(2))
df['måned'] = df['deployed_timestamp'].dt.isocalendar().year.astype('str') + '-' + df['måned']
df['år'] = df['deployed_timestamp'].dt.isocalendar().year
df['app'] = df['application']

In [ ]:
år = df.groupby(['uke', 'år']).size().reset_index(name='antall') \
       .groupby('år').agg(snitt_deploys_per_uke=('antall', 'mean')).reset_index()
år['snitt_deploys_per_uke'] = år['snitt_deploys_per_uke'].round(0)
år = år[år['år']>2014]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
figure(figsize=(14, 6), dpi=80)
 
with plt.xkcd():
    plt.bar(år['år'], år['snitt_deploys_per_uke'], color='grey')
    plt.title('Average weekly deploys to production in NAV')
    plt.xlabel('year')
    plt.ylabel('deploys per week')

     
    plt.show()